In [1]:
# Here I will be rethinkg how to create the trainData, valData and testData
# i will create a new dataset trainData2
# after the hyperparameter tuning: I have good final dimension and learning rate
# then in the trainData2 i will try to give as much supervision edges and supervision labels as possible
# that way model can learn better as it has more data to train on

In [2]:
# giving as much supervision edges as possible mean: from train_dti
# i will use 90% edges for supervision and 10% for message passing

In [3]:
# first redicover how I created the heterogeneous graphs 

In [1]:
import pandas as pd
import glob
import os
import torch
from torch import Tensor
import numpy as np
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, to_hetero
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [2]:
# this function will be used to build drug_feat
def to_float(value):
    try:
        return float(value)
    except ValueError:
        return float('nan')

In [3]:
# creating drug_feat
file_path = './NE Dataset/drug_attr.csv'
dfDrug = pd.read_csv(file_path)
dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)
dfDrugFeat.replace('#NAME?', np.nan, inplace=True) # replacing the missing values with 0
dfDrugFeat.fillna(0, inplace=True)
dfDrugFeat = dfDrugFeat.applymap(to_float)
dfDrugFeat.replace([np.inf], 0, inplace=True) # replacing the inf value with zero
scaler = MinMaxScaler(feature_range=(-1, 1)) # NORMALIZE HERE
dfDrugFeat = scaler.fit_transform(dfDrugFeat)
drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)

/tmp/ipykernel_1846610/3469342505.py:3: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [4]:
# creating target_feat
file_path = './NE Dataset/protein_attr.csv'
dfTarget = pd.read_csv(file_path)
dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)
target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)

In [5]:
# unique drug and unique target mapping
uniqueDrug = dfDrug['1'].unique()
uniqueDrug = pd.DataFrame(data={
    'DrugId': uniqueDrug,
    'mappedId': pd.RangeIndex(len(uniqueDrug)),
})
uniqueTraget = dfTarget['#'].unique()
uniqueTraget = pd.DataFrame(data={
    'TargetId': uniqueTraget,
    'mappedId': pd.RangeIndex(len(uniqueTraget)),
})

In [6]:
# creating edge_index_drug_to_drug
file_path = './NE Dataset/train_ddi.csv'
dfDDI = pd.read_csv(file_path)
source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                        left_on='Drug', right_on='DrugId', how='left')
source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)
destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                              left_on='Target', right_on='DrugId', how='left')
destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)
edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)

In [7]:
# creating edge_index_target_to_target
file_path = './NE Dataset/train_ppi.csv'
dfPPI = pd.read_csv(file_path)
source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                            left_on='Drug', right_on='TargetId', how='left')
source_target_id = torch.from_numpy(source_target_id['mappedId'].values)
destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                left_on='Target', right_on='TargetId', how='left')
destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)
edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)

In [8]:
file_path = './NE Dataset/train_dti.csv'

In [9]:
dfDTI = pd.read_csv(file_path)
dfDTIpos = dfDTI[dfDTI['Label'] == 1]
dfDTIneg = dfDTI[dfDTI['Label'] == 0]

In [10]:
pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                                left_on='Drug', right_on='DrugId', how='left')
pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)
pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                  left_on='Target', right_on='TargetId', how='left')
pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)
edge_index_pos_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0) # edge_index_pos_drug_to_target

In [11]:
neg_interact_drug_id = pd.merge(dfDTIneg['Drug'], uniqueDrug,
                                left_on='Drug', right_on='DrugId', how='left')
neg_interact_drug_id = torch.from_numpy(neg_interact_drug_id['mappedId'].values)
neg_interact_target_id = pd.merge(dfDTIneg['Target'], uniqueTraget,
                                  left_on='Target', right_on='TargetId', how='left')
neg_interact_target_id = torch.from_numpy(neg_interact_target_id['mappedId'].values)
edge_index_neg_drug_to_target = torch.stack([neg_interact_drug_id, neg_interact_target_id], dim=0) # edge_index_neg_drug_to_target

In [12]:
pos_edge_label = dfDTIpos['Label']
pos_edge_label = torch.tensor(pos_edge_label.values) # pos_edge_label

In [13]:
neg_edge_label = dfDTIneg['Label']
neg_edge_label = torch.tensor(neg_edge_label.values) # neg_edge_label

In [14]:
# this function will be used to create edge_index_supervision and edge_label_supervision
def split_edges(edge_index, edge_label, split_ratio=0.7):
    num_edges = edge_index.size(1)
    random_permutation = torch.randperm(num_edges)
    split_edge = int(split_ratio * num_edges)
    edge_index_meg_pass = edge_index[:, random_permutation[:split_edge]]
    edge_index_supervision = edge_index[:, random_permutation[split_edge: ]]
    edge_label_supervision = edge_label[random_permutation[split_edge: ]]
    return edge_index_meg_pass, edge_index_supervision, edge_label_supervision

In [15]:
# creating edge_index_supervision and edge_label_supervision
edge_index_meg_pass, pos_edge_index_supervision, pos_edge_label_supervision = split_edges(edge_index_pos_drug_to_target, pos_edge_label)
dropped_edge_index, neg_edge_index_supervision, neg_edge_label_supervision = split_edges(edge_index_neg_drug_to_target, neg_edge_label)
edge_index_supervision = torch.cat((pos_edge_index_supervision, neg_edge_index_supervision), dim=1)
edge_label_supervision = torch.cat((pos_edge_label_supervision, neg_edge_label_supervision), dim=0)
random_permutation = torch.randperm(1424)
edge_index_supervision = edge_index_supervision[:, random_permutation]
edge_label_supervision = edge_label_supervision[random_permutation]

In [16]:
# creating trainData
# adding node and features of drug
trainData = HeteroData()
trainData['drug'].node_id = torch.arange(drug_feat.size(0))
trainData['drug'].x = drug_feat

In [17]:
# adding node and features of target
trainData['target'].node_id = torch.arange(target_feat.size(0))
trainData['target'].x = target_feat

In [18]:
# adding all the edges which will be used for message passing to the heterogeneous data
trainData["drug", "interact", "target"].edge_index = edge_index_meg_pass
trainData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
trainData["target", "similar", "target"].edge_index = edge_index_target_to_target

In [19]:
trainData = T.ToUndirected()(trainData) # making all the message passing edges undirected

In [20]:
# adding edge index and edge label for supervision to the data
trainData["drug", "interact", "target"].edge_label = edge_label_supervision
trainData["drug", "interact", "target"].edge_label_index = edge_index_supervision

In [21]:
trainData

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 1659],
    edge_label=[1424],
    edge_label_index=[2, 1424],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 1659] }
)

In [26]:
# Now i will revisit how I created valData

In [22]:
# creating drug_feat
file_path = './NE Dataset/drug_attr.csv'
dfDrug = pd.read_csv(file_path)
dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)
dfDrugFeat.replace('#NAME?', np.nan, inplace=True) # replacing the missing values with 0
dfDrugFeat.fillna(0, inplace=True)
dfDrugFeat = dfDrugFeat.applymap(to_float)
dfDrugFeat.replace([np.inf], 0, inplace=True) # replacing the inf value with zero
scaler = MinMaxScaler(feature_range=(-1, 1)) # NORMALIZE HERE
dfDrugFeat = scaler.fit_transform(dfDrugFeat)
drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)
# creating target_feat
file_path = './NE Dataset/protein_attr.csv'
dfTarget = pd.read_csv(file_path)
dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)
target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)
# unique drug and unique target mapping
uniqueDrug = dfDrug['1'].unique()
uniqueDrug = pd.DataFrame(data={
    'DrugId': uniqueDrug,
    'mappedId': pd.RangeIndex(len(uniqueDrug)),
})
uniqueTraget = dfTarget['#'].unique()
uniqueTraget = pd.DataFrame(data={
    'TargetId': uniqueTraget,
    'mappedId': pd.RangeIndex(len(uniqueTraget)),
})

/tmp/ipykernel_1846610/3262938377.py:3: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [23]:
# creating edge_index_drug_to_drug
file_path = './NE Dataset/train_ddi.csv'
dfDDI = pd.read_csv(file_path)
source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                          left_on='Drug', right_on='DrugId', how='left')
source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)
destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                               left_on='Target', right_on='DrugId', how='left')
destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)
edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)

In [24]:
 # creating edge_index_target_to_target
file_path = './NE Dataset/train_ppi.csv'
dfPPI = pd.read_csv(file_path)
source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                            left_on='Drug', right_on='TargetId', how='left')
source_target_id = torch.from_numpy(source_target_id['mappedId'].values)
destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                left_on='Target', right_on='TargetId', how='left')
destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)
edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)

In [25]:
# creating edge_index_pos_drug_to_target [these edges will be used for message passing]
file_path = './NE Dataset/train_dti.csv'
dfDTI = pd.read_csv(file_path)
dfDTIpos = dfDTI[dfDTI['Label'] == 1]
pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                                left_on='Drug', right_on='DrugId', how='left')
pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)
pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                      left_on='Target', right_on='TargetId', how='left')
pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)
edge_index_pos_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0)

In [26]:
# creating supervision edge and label
file_path = './NE Dataset/val_dti.csv'
dfvalDTI = pd.read_csv(file_path)
edge_index_supervision_drug = pd.merge(dfvalDTI['Drug'], uniqueDrug,
                                       left_on='Drug', right_on='DrugId', how='left')
edge_index_supervision_drug = torch.from_numpy(edge_index_supervision_drug['mappedId'].values)
edge_index_supervision_target = pd.merge(dfvalDTI['Target'], uniqueTraget,
                                        left_on='Target', right_on='TargetId', how='left')
edge_index_supervision_target = torch.from_numpy(edge_index_supervision_target['mappedId'].values)
edge_index_supervision = torch.stack([edge_index_supervision_drug, edge_index_supervision_target], dim=0)
edge_label_supervision = dfvalDTI['Label']
edge_label_supervision = torch.tensor(edge_label_supervision.values)
random_permutationVal = torch.randperm(948)
edge_index_supervision = edge_index_supervision[:, random_permutationVal]
edge_label_supervision = edge_label_supervision[random_permutationVal]

In [27]:
# creating the heterogeneous graph valData
valData = HeteroData()

In [28]:
# adding node and features of drug
valData['drug'].node_id = torch.arange(drug_feat.size(0))
valData['drug'].x = drug_feat

In [29]:
# adding node and features of target
valData['target'].node_id = torch.arange(target_feat.size(0))
valData['target'].x = target_feat

In [30]:
# adding all the edges which will be used for message passing to the heterogeneous data
valData["drug", "interact", "target"].edge_index = edge_index_pos_drug_to_target
valData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
valData["target", "similar", "target"].edge_index = edge_index_target_to_target

In [31]:
# making all the edges undirected
valData = T.ToUndirected()(valData)

In [32]:
# adding edge index and edge label for supervision to the data
valData["drug", "interact", "target"].edge_label = edge_label_supervision
valData["drug", "interact", "target"].edge_label_index = edge_index_supervision

In [33]:
valData

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 2371],
    edge_label=[948],
    edge_label_index=[2, 948],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 2371] }
)

In [49]:
# now I will create trainData2
# in this trainData2 -> 90% edges from train_dti will be used to for supervision
# so split_ratio function er ratio = 0.1 set kore train_dti er pos gulu devide korte hbe

In [ ]:
# for trainData2
# (drug, interact, target)={
# edge_index=[2, 237]    
# edge_label = [5216],
# edge_label_index = [2, 5216],
#}

In [49]:
# creation of trainData2

In [34]:
# creating drug_feat
file_path = './NE Dataset/drug_attr.csv'
dfDrug = pd.read_csv(file_path)
dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)
dfDrugFeat.replace('#NAME?', np.nan, inplace=True) # replacing the missing values with 0
dfDrugFeat.fillna(0, inplace=True)
dfDrugFeat = dfDrugFeat.applymap(to_float)
dfDrugFeat.replace([np.inf], 0, inplace=True) # replacing the inf value with zero
scaler = MinMaxScaler(feature_range=(-1, 1)) # NORMALIZE HERE
dfDrugFeat = scaler.fit_transform(dfDrugFeat)
drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)
# creating target_feat
file_path = './NE Dataset/protein_attr.csv'
dfTarget = pd.read_csv(file_path)
dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)
target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)
# unique drug and unique target mapping
uniqueDrug = dfDrug['1'].unique()
uniqueDrug = pd.DataFrame(data={
    'DrugId': uniqueDrug,
    'mappedId': pd.RangeIndex(len(uniqueDrug)),
})
uniqueTraget = dfTarget['#'].unique()
uniqueTraget = pd.DataFrame(data={
    'TargetId': uniqueTraget,
    'mappedId': pd.RangeIndex(len(uniqueTraget)),
})

/tmp/ipykernel_1846610/3262938377.py:3: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [35]:
# creating edge_index_drug_to_drug
file_path = './NE Dataset/train_ddi.csv'
dfDDI = pd.read_csv(file_path)
source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                          left_on='Drug', right_on='DrugId', how='left')
source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)
destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                               left_on='Target', right_on='DrugId', how='left')
destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)
edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)

In [36]:
# creating edge_index_target_to_target
file_path = './NE Dataset/train_ppi.csv'
dfPPI = pd.read_csv(file_path)
source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                            left_on='Drug', right_on='TargetId', how='left')
source_target_id = torch.from_numpy(source_target_id['mappedId'].values)
destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                left_on='Target', right_on='TargetId', how='left')
destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)
edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)

In [37]:
file_path = './NE Dataset/train_dti.csv'

In [38]:
dfDTI = pd.read_csv(file_path)

In [39]:
dfDTIpos = dfDTI[dfDTI['Label'] == 1]

In [40]:
dfDTIneg = dfDTI[dfDTI['Label'] == 0]

In [41]:
pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                                left_on='Drug', right_on='DrugId', how='left')
pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)
pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                 left_on='Target', right_on='TargetId', how='left')
pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)

In [42]:
edge_index_pos_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0) # edge_index_pos_drug_to_target

In [43]:
edge_index_pos_drug_to_target.size()

torch.Size([2, 2371])

In [44]:
neg_interact_drug_id = pd.merge(dfDTIneg['Drug'], uniqueDrug,
                                left_on='Drug', right_on='DrugId', how='left')
neg_interact_drug_id = torch.from_numpy(neg_interact_drug_id['mappedId'].values)
neg_interact_target_id = pd.merge(dfDTIneg['Target'], uniqueTraget,
                                  left_on='Target', right_on='TargetId', how='left')
neg_interact_target_id = torch.from_numpy(neg_interact_target_id['mappedId'].values)
edge_index_neg_drug_to_target = torch.stack([neg_interact_drug_id, neg_interact_target_id], dim=0)

In [45]:
edge_index_neg_drug_to_target.size()

torch.Size([2, 2371])

In [46]:
pos_edge_label = dfDTIpos['Label']
pos_edge_label = torch.tensor(pos_edge_label.values)
neg_edge_label = dfDTIneg['Label']
neg_edge_label = torch.tensor(neg_edge_label.values)

In [47]:
# this function will be used to create edge_index_supervision and edge_label_supervision
def split_edges(edge_index, edge_label, split_ratio=0.7):
    num_edges = edge_index.size(1)
    random_permutation = torch.randperm(num_edges)
    split_edge = int(split_ratio * num_edges)
    edge_index_meg_pass = edge_index[:, random_permutation[:split_edge]]
    edge_index_supervision = edge_index[:, random_permutation[split_edge: ]]
    edge_label_supervision = edge_label[random_permutation[split_edge: ]]
    return edge_index_meg_pass, edge_index_supervision, edge_label_supervision

In [48]:
edge_index_meg_pass, pos_edge_index_supervision, pos_edge_label_supervision = split_edges(edge_index_pos_drug_to_target, pos_edge_label, 0.1)

In [51]:
# pos_edge_index_supervision.size()

In [52]:
dropped_edge_index, neg_edge_index_supervision, neg_edge_label_supervision = split_edges(edge_index_neg_drug_to_target, neg_edge_label, 0.1)

In [71]:
# neg_edge_index_supervision.size()

In [53]:
edge_index_supervision_from_train_dti = torch.cat((pos_edge_index_supervision, neg_edge_index_supervision), dim=1)
edge_label_supervision_from_train_dti = torch.cat((pos_edge_label_supervision, neg_edge_label_supervision), dim=0)

In [55]:
# edge_label_supervision_from_train_dti.size()

In [56]:
edge_index_supervision_from_train_dti

tensor([[ 449,  341, 1481,  ...,  824,  612,  784],
        [ 288,    9,  787,  ...,  190,  321,  139]])

In [57]:
edge_label_supervision_from_train_dti

tensor([1, 1, 1,  ..., 0, 0, 0])

In [59]:
random_permutation = torch.randperm(4268)

In [60]:
edge_index_supervision_from_train_dti = edge_index_supervision_from_train_dti[:, random_permutation]

In [61]:
edge_label_supervision_from_train_dti = edge_label_supervision_from_train_dti[random_permutation]

In [62]:
edge_index_supervision_from_train_dti

tensor([[ 858,  805,  468,  ...,  925,  716,  716],
        [ 942,  676,  565,  ...,  662,  372, 1329]])

In [63]:
edge_label_supervision_from_train_dti

tensor([1, 0, 0,  ..., 1, 1, 0])

In [64]:
 # creating supervision edge and label from val_dti
file_path = './NE Dataset/val_dti.csv'
dfvalDTI = pd.read_csv(file_path)

In [65]:
edge_index_supervision_drug = pd.merge(dfvalDTI['Drug'], uniqueDrug,
                                       left_on='Drug', right_on='DrugId', how='left')
edge_index_supervision_drug = torch.from_numpy(edge_index_supervision_drug['mappedId'].values)
edge_index_supervision_target = pd.merge(dfvalDTI['Target'], uniqueTraget,
                                         left_on='Target', right_on='TargetId', how='left')
edge_index_supervision_target = torch.from_numpy(edge_index_supervision_target['mappedId'].values)
edge_index_supervision = torch.stack([edge_index_supervision_drug, edge_index_supervision_target], dim=0)
edge_label_supervision = dfvalDTI['Label']
edge_label_supervision = torch.tensor(edge_label_supervision.values)
random_permutationVal = torch.randperm(948)
edge_index_supervision_from_val_dti = edge_index_supervision[:, random_permutationVal]
edge_label_supervision_from_val_dti = edge_label_supervision[random_permutationVal]

In [68]:
edge_index_supervision_from_train_dti.size()

torch.Size([2, 4268])

In [70]:
edge_index_supervision_from_train_dti

tensor([[ 858,  805,  468,  ...,  925,  716,  716],
        [ 942,  676,  565,  ...,  662,  372, 1329]])

In [66]:
edge_index_supervision_from_val_dti.size()

torch.Size([2, 948])

In [71]:
edge_index_supervision_from_val_dti

tensor([[ 456,  528,  725,  ...,  272,  143, 1266],
        [ 176,  111, 1293,  ..., 1284, 1055,  371]])

In [ ]:
# concatenate edge_index_supervision_from_train_dti and edge_index_supervision_from_val_dti
# and label also
# define trainData2 and do the rest

In [72]:
edge_index_supervision = torch.cat((edge_index_supervision_from_train_dti, edge_index_supervision_from_val_dti), dim=1)

In [78]:
# edge_index_supervision.size()

In [76]:
edge_label_supervision = torch.cat((edge_label_supervision_from_train_dti, edge_label_supervision_from_val_dti), dim=0)

In [79]:
# edge_label_supervision.size()

In [80]:
trainData2 = HeteroData()

In [81]:
# adding node and features of drug
trainData2['drug'].node_id = torch.arange(drug_feat.size(0))
trainData2['drug'].x = drug_feat

In [82]:
# adding node and features of target
trainData2['target'].node_id = torch.arange(target_feat.size(0))
trainData2['target'].x = target_feat

In [85]:
edge_index_meg_pass.size()

torch.Size([2, 237])

In [88]:
# adding all the edges which will be used for message passing to the heterogeneous data
trainData2["drug", "interact", "target"].edge_index = edge_index_meg_pass
trainData2["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
trainData2["target", "similar", "target"].edge_index = edge_index_target_to_target

In [90]:
trainData2 = T.ToUndirected()(trainData2) # making all the message passing edges undirected

In [92]:
# adding edge index and edge label for supervision to the data
trainData2["drug", "interact", "target"].edge_label = edge_label_supervision
trainData2["drug", "interact", "target"].edge_label_index = edge_index_supervision

In [95]:
valData

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 2371],
    edge_label=[948],
    edge_label_index=[2, 948],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 2371] }
)

In [93]:
trainData2

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 237],
    edge_label=[5216],
    edge_label_index=[2, 5216],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 237] }
)

In [ ]:
# adding node and features of drug
    trainData = HeteroData()
    trainData['drug'].node_id = torch.arange(drug_feat.size(0))
    trainData['drug'].x = drug_feat
    # adding node and features of target
    trainData['target'].node_id = torch.arange(target_feat.size(0))
    trainData['target'].x = target_feat
    # adding all the edges which will be used for message passing to the heterogeneous data
    trainData["drug", "interact", "target"].edge_index = edge_index_meg_pass
    trainData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
    trainData["target", "similar", "target"].edge_index = edge_index_target_to_target
    trainData = T.ToUndirected()(trainData) # making all the message passing edges undirected
    # adding edge index and edge label for supervision to the data
    trainData["drug", "interact", "target"].edge_label = edge_label_supervision
    trainData["drug", "interact", "target"].edge_label_index = edge_index_supervision

In [ ]:
 # creating supervision edge and label
file_path = './NE Dataset/val_dti.csv'
dfvalDTI = pd.read_csv(file_path)
edge_index_supervision_drug = pd.merge(dfvalDTI['Drug'], uniqueDrug,
                                       left_on='Drug', right_on='DrugId', how='left')
edge_index_supervision_drug = torch.from_numpy(edge_index_supervision_drug['mappedId'].values)
edge_index_supervision_target = pd.merge(dfvalDTI['Target'], uniqueTraget,
                                         left_on='Target', right_on='TargetId', how='left')
edge_index_supervision_target = torch.from_numpy(edge_index_supervision_target['mappedId'].values)
edge_index_supervision = torch.stack([edge_index_supervision_drug, edge_index_supervision_target], dim=0)
edge_label_supervision = dfvalDTI['Label']
edge_label_supervision = torch.tensor(edge_label_supervision.values)
random_permutationVal = torch.randperm(948)
edge_index_supervision = edge_index_supervision[:, random_permutationVal]
edge_label_supervision = edge_label_supervision[random_permutationVal]

In [1]:
import pandas as pd
import glob
import os
import torch
from torch import Tensor
import numpy as np
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, to_hetero
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

In [2]:
# this function will be used to build drug_feat
def to_float(value):
    try:
        return float(value)
    except ValueError:
        return float('nan')

In [3]:
# this function will be used to create edge_index_supervision and edge_label_supervision
def split_edges(edge_index, edge_label, split_ratio=0.7):
    num_edges = edge_index.size(1)
    random_permutation = torch.randperm(num_edges)
    split_edge = int(split_ratio * num_edges)
    edge_index_meg_pass = edge_index[:, random_permutation[:split_edge]]
    edge_index_supervision = edge_index[:, random_permutation[split_edge: ]]
    edge_label_supervision = edge_label[random_permutation[split_edge: ]]
    return edge_index_meg_pass, edge_index_supervision, edge_label_supervision

In [4]:
# I will creat a function to build trainData

In [7]:
def create_trainData():
    # creating drug_feat
    file_path = './NE Dataset/drug_attr.csv'
    dfDrug = pd.read_csv(file_path)
    dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)
    dfDrugFeat.replace('#NAME?', np.nan, inplace=True) # replacing the missing values with 0
    dfDrugFeat.fillna(0, inplace=True)
    dfDrugFeat = dfDrugFeat.applymap(to_float)
    dfDrugFeat.replace([np.inf], 0, inplace=True) # replacing the inf value with zero
    scaler = MinMaxScaler(feature_range=(-1, 1)) # NORMALIZE HERE
    dfDrugFeat = scaler.fit_transform(dfDrugFeat)
    drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)
    # creating target_feat
    file_path = './NE Dataset/protein_attr.csv'
    dfTarget = pd.read_csv(file_path)
    dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)
    target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)
    # unique drug and unique target mapping
    uniqueDrug = dfDrug['1'].unique()
    uniqueDrug = pd.DataFrame(data={
        'DrugId': uniqueDrug,
        'mappedId': pd.RangeIndex(len(uniqueDrug)),
    })
    uniqueTraget = dfTarget['#'].unique()
    uniqueTraget = pd.DataFrame(data={
        'TargetId': uniqueTraget,
        'mappedId': pd.RangeIndex(len(uniqueTraget)),
    })
    # creating edge_index_drug_to_drug
    file_path = './NE Dataset/train_ddi.csv'
    dfDDI = pd.read_csv(file_path)
    source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                             left_on='Drug', right_on='DrugId', how='left')
    source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)
    destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                                  left_on='Target', right_on='DrugId', how='left')
    destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)
    edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)
    # creating edge_index_target_to_target
    file_path = './NE Dataset/train_ppi.csv'
    dfPPI = pd.read_csv(file_path)
    source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                               left_on='Drug', right_on='TargetId', how='left')
    source_target_id = torch.from_numpy(source_target_id['mappedId'].values)
    destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                    left_on='Target', right_on='TargetId', how='left')
    destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)
    edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)
    # creating edge_index_pos_drug_to_target, edge_index_neg_drug_to_target, pos_edge_label, neg_edge_label
    file_path = './NE Dataset/train_dti.csv'
    dfDTI = pd.read_csv(file_path)
    dfDTIpos = dfDTI[dfDTI['Label'] == 1]
    dfDTIneg = dfDTI[dfDTI['Label'] == 0]
    pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                                   left_on='Drug', right_on='DrugId', how='left')
    pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)
    pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                      left_on='Target', right_on='TargetId', how='left')
    pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)
    edge_index_pos_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0) # edge_index_pos_drug_to_target
    neg_interact_drug_id = pd.merge(dfDTIneg['Drug'], uniqueDrug,
                                   left_on='Drug', right_on='DrugId', how='left')
    neg_interact_drug_id = torch.from_numpy(neg_interact_drug_id['mappedId'].values)
    neg_interact_target_id = pd.merge(dfDTIneg['Target'], uniqueTraget,
                                     left_on='Target', right_on='TargetId', how='left')
    neg_interact_target_id = torch.from_numpy(neg_interact_target_id['mappedId'].values)
    edge_index_neg_drug_to_target = torch.stack([neg_interact_drug_id, neg_interact_target_id], dim=0) # edge_index_neg_drug_to_target
    pos_edge_label = dfDTIpos['Label']
    pos_edge_label = torch.tensor(pos_edge_label.values) # pos_edge_label
    neg_edge_label = dfDTIneg['Label']
    neg_edge_label = torch.tensor(neg_edge_label.values) # neg_edge_label
    # creating edge_index_supervision and edge_label_supervision
    edge_index_meg_pass, pos_edge_index_supervision, pos_edge_label_supervision = split_edges(edge_index_pos_drug_to_target, pos_edge_label)
    dropped_edge_index, neg_edge_index_supervision, neg_edge_label_supervision = split_edges(edge_index_neg_drug_to_target, neg_edge_label)
    edge_index_supervision = torch.cat((pos_edge_index_supervision, neg_edge_index_supervision), dim=1)
    edge_label_supervision = torch.cat((pos_edge_label_supervision, neg_edge_label_supervision), dim=0)
    random_permutation = torch.randperm(1424)
    edge_index_supervision = edge_index_supervision[:, random_permutation]
    edge_label_supervision = edge_label_supervision[random_permutation]
    # creating trainData
    # adding node and features of drug
    trainData = HeteroData()
    trainData['drug'].node_id = torch.arange(drug_feat.size(0))
    trainData['drug'].x = drug_feat
    # adding node and features of target
    trainData['target'].node_id = torch.arange(target_feat.size(0))
    trainData['target'].x = target_feat
    # adding all the edges which will be used for message passing to the heterogeneous data
    trainData["drug", "interact", "target"].edge_index = edge_index_meg_pass
    trainData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
    trainData["target", "similar", "target"].edge_index = edge_index_target_to_target
    trainData = T.ToUndirected()(trainData) # making all the message passing edges undirected
    # adding edge index and edge label for supervision to the data
    trainData["drug", "interact", "target"].edge_label = edge_label_supervision
    trainData["drug", "interact", "target"].edge_label_index = edge_index_supervision
    return trainData

In [8]:
trainData = create_trainData()

/tmp/ipykernel_173621/1833806588.py:4: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [9]:
print(trainData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 1659],
    edge_label=[1424],
    edge_label_index=[2, 1424],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 1659] }
)


In [49]:
# I will create a function to build valData

In [10]:
def create_valData():
    # creating drug_feat
    file_path = './NE Dataset/drug_attr.csv'
    dfDrug = pd.read_csv(file_path)
    dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)
    dfDrugFeat.replace('#NAME?', np.nan, inplace=True) # replacing the missing values with 0
    dfDrugFeat.fillna(0, inplace=True)
    dfDrugFeat = dfDrugFeat.applymap(to_float)
    dfDrugFeat.replace([np.inf], 0, inplace=True) # replacing the inf value with zero
    scaler = MinMaxScaler(feature_range=(-1, 1)) # NORMALIZE HERE
    dfDrugFeat = scaler.fit_transform(dfDrugFeat)
    drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)
    # creating target_feat
    file_path = './NE Dataset/protein_attr.csv'
    dfTarget = pd.read_csv(file_path)
    dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)
    target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)
    # unique drug and unique target mapping
    uniqueDrug = dfDrug['1'].unique()
    uniqueDrug = pd.DataFrame(data={
        'DrugId': uniqueDrug,
        'mappedId': pd.RangeIndex(len(uniqueDrug)),
    })
    uniqueTraget = dfTarget['#'].unique()
    uniqueTraget = pd.DataFrame(data={
        'TargetId': uniqueTraget,
        'mappedId': pd.RangeIndex(len(uniqueTraget)),
    })
    # creating edge_index_drug_to_drug
    file_path = './NE Dataset/train_ddi.csv'
    dfDDI = pd.read_csv(file_path)
    source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                             left_on='Drug', right_on='DrugId', how='left')
    source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)
    destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                                  left_on='Target', right_on='DrugId', how='left')
    destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)
    edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)
    # creating edge_index_target_to_target
    file_path = './NE Dataset/train_ppi.csv'
    dfPPI = pd.read_csv(file_path)
    source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                               left_on='Drug', right_on='TargetId', how='left')
    source_target_id = torch.from_numpy(source_target_id['mappedId'].values)
    destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                    left_on='Target', right_on='TargetId', how='left')
    destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)
    edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)
    # creating edge_index_pos_drug_to_target [these edges will be used for message passing]
    file_path = './NE Dataset/train_dti.csv'
    dfDTI = pd.read_csv(file_path)
    dfDTIpos = dfDTI[dfDTI['Label'] == 1]
    pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                                   left_on='Drug', right_on='DrugId', how='left')
    pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)
    pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                      left_on='Target', right_on='TargetId', how='left')
    pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)
    edge_index_pos_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0)
    # creating supervision edge and label
    file_path = './NE Dataset/val_dti.csv'
    dfvalDTI = pd.read_csv(file_path)
    edge_index_supervision_drug = pd.merge(dfvalDTI['Drug'], uniqueDrug,
                                          left_on='Drug', right_on='DrugId', how='left')
    edge_index_supervision_drug = torch.from_numpy(edge_index_supervision_drug['mappedId'].values)
    edge_index_supervision_target = pd.merge(dfvalDTI['Target'], uniqueTraget,
                                            left_on='Target', right_on='TargetId', how='left')
    edge_index_supervision_target = torch.from_numpy(edge_index_supervision_target['mappedId'].values)
    edge_index_supervision = torch.stack([edge_index_supervision_drug, edge_index_supervision_target], dim=0)
    edge_label_supervision = dfvalDTI['Label']
    edge_label_supervision = torch.tensor(edge_label_supervision.values)
    random_permutationVal = torch.randperm(948)
    edge_index_supervision = edge_index_supervision[:, random_permutationVal]
    edge_label_supervision = edge_label_supervision[random_permutationVal]
    # creating the heterogeneous graph valData
    valData = HeteroData()
    # adding node and features of drug
    valData['drug'].node_id = torch.arange(drug_feat.size(0))
    valData['drug'].x = drug_feat
    # adding node and features of target
    valData['target'].node_id = torch.arange(target_feat.size(0))
    valData['target'].x = target_feat
    # adding all the edges which will be used for message passing to the heterogeneous data
    valData["drug", "interact", "target"].edge_index = edge_index_pos_drug_to_target
    valData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
    valData["target", "similar", "target"].edge_index = edge_index_target_to_target
    # making all the edges undirected
    valData = T.ToUndirected()(valData)
    # adding edge index and edge label for supervision to the data
    valData["drug", "interact", "target"].edge_label = edge_label_supervision
    valData["drug", "interact", "target"].edge_label_index = edge_index_supervision
    return valData

In [11]:
valData = create_valData()

/tmp/ipykernel_173621/1106223255.py:4: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [12]:
print(valData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 2371],
    edge_label=[948],
    edge_label_index=[2, 948],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 2371] }
)


In [61]:
# I will create a function to create testData

In [13]:
def create_testData():
    # creating drug_feat
    file_path = './NE Dataset/drug_attr.csv'
    dfDrug = pd.read_csv(file_path)
    dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)
    dfDrugFeat.replace('#NAME?', np.nan, inplace=True) # replacing the missing values with 0
    dfDrugFeat.fillna(0, inplace=True)
    dfDrugFeat = dfDrugFeat.applymap(to_float)
    dfDrugFeat.replace([np.inf], 0, inplace=True) # replacing the inf value with zero
    scaler = MinMaxScaler(feature_range=(-1, 1)) # NORMALIZE HERE
    dfDrugFeat = scaler.fit_transform(dfDrugFeat)
    drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)
    # creating target_feat
    file_path = './NE Dataset/protein_attr.csv'
    dfTarget = pd.read_csv(file_path)
    dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)
    target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)
    # unique drug and unique target mapping
    uniqueDrug = dfDrug['1'].unique()
    uniqueDrug = pd.DataFrame(data={
        'DrugId': uniqueDrug,
        'mappedId': pd.RangeIndex(len(uniqueDrug)),
    })
    uniqueTraget = dfTarget['#'].unique()
    uniqueTraget = pd.DataFrame(data={
        'TargetId': uniqueTraget,
        'mappedId': pd.RangeIndex(len(uniqueTraget)),
    })
    # creating edge_index_drug_to_drug
    file_path = './NE Dataset/train_ddi.csv'
    dfDDI = pd.read_csv(file_path)
    source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                             left_on='Drug', right_on='DrugId', how='left')
    source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)
    destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                                  left_on='Target', right_on='DrugId', how='left')
    destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)
    edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)
    # creating edge_index_target_to_target
    file_path = './NE Dataset/train_ppi.csv'
    dfPPI = pd.read_csv(file_path)
    source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                               left_on='Drug', right_on='TargetId', how='left')
    source_target_id = torch.from_numpy(source_target_id['mappedId'].values)
    destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                    left_on='Target', right_on='TargetId', how='left')
    destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)
    edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)
    # create edge_index_pos_drug_to_target_train
    file_path = './NE Dataset/train_dti.csv'
    dfDTI = pd.read_csv(file_path)
    dfDTIpos = dfDTI[dfDTI['Label'] == 1]
    pos_interact_drug_id_train = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                                          left_on='Drug', right_on='DrugId', how='left')
    pos_interact_drug_id_train = torch.from_numpy(pos_interact_drug_id_train['mappedId'].values)
    pos_interact_target_id_train = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                            left_on='Target', right_on='TargetId', how='left')
    pos_interact_target_id_train = torch.from_numpy(pos_interact_target_id_train['mappedId'].values)
    edge_index_pos_drug_to_target_train = torch.stack([pos_interact_drug_id_train, pos_interact_target_id_train], dim=0)
    # create edge_index_pos_drug_to_target_val
    file_path = './NE Dataset/val_dti.csv'
    dfvalDTI = pd.read_csv(file_path)
    dfvalDTIpos = dfvalDTI[dfvalDTI['Label'] == 1]
    pos_interact_drug_id_val = pd.merge(dfvalDTIpos['Drug'], uniqueDrug,
                                       left_on='Drug', right_on='DrugId', how='left')
    pos_interact_drug_id_val = torch.from_numpy(pos_interact_drug_id_val['mappedId'].values)
    pos_interact_target_id_val = pd.merge(dfvalDTIpos['Target'], uniqueTraget,
                                       left_on='Target', right_on='TargetId', how='left')
    pos_interact_target_id_val = torch.from_numpy(pos_interact_target_id_val['mappedId'].values)
    edge_index_pos_drug_to_target_val = torch.stack([pos_interact_drug_id_val, pos_interact_target_id_val], dim=0)
    # create edge_index_pos_drug_to_target
    edge_index_pos_drug_to_target = torch.cat((edge_index_pos_drug_to_target_train, edge_index_pos_drug_to_target_val), dim=1)
    # creating supervision edge and label
    file_path = './NE Dataset/test_dti.csv'
    dftestDTI = pd.read_csv(file_path)
    edge_index_supervision_drug = pd.merge(dftestDTI['Drug'], uniqueDrug,
                                           left_on='Drug', right_on='DrugId', how='left')
    edge_index_supervision_drug = torch.from_numpy(edge_index_supervision_drug['mappedId'].values)
    edge_index_supervision_target = pd.merge(dftestDTI['Target'], uniqueTraget,
                                        left_on='Target', right_on='TargetId', how='left')
    edge_index_supervision_target = torch.from_numpy(edge_index_supervision_target['mappedId'].values)
    edge_index_supervision = torch.stack([edge_index_supervision_drug, edge_index_supervision_target], dim=0)
    edge_label_supervision = dftestDTI['Label']
    edge_label_supervision = torch.tensor(edge_label_supervision.values)
    random_permutationTest = torch.randperm(2214)
    edge_index_supervision = edge_index_supervision[:, random_permutationTest]
    edge_label_supervision = edge_label_supervision[random_permutationTest]
    # create the heterogeneous graph testData
    testData = HeteroData()
    # adding node and features of drug
    testData['drug'].node_id = torch.arange(drug_feat.size(0))
    testData['drug'].x = drug_feat
    # adding node and features of target
    testData['target'].node_id = torch.arange(target_feat.size(0))
    testData['target'].x = target_feat
    # adding all the edges which will be used for message passing to the heterogeneous data
    testData["drug", "interact", "target"].edge_index = edge_index_pos_drug_to_target
    testData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
    testData["target", "similar", "target"].edge_index = edge_index_target_to_target
    # making all the edges undirected
    testData = T.ToUndirected()(testData)
    # adding edge index and edge label for supervision to the data
    testData["drug", "interact", "target"].edge_label = edge_label_supervision
    testData["drug", "interact", "target"].edge_label_index = edge_index_supervision
    return testData

In [14]:
testData = create_testData()

/tmp/ipykernel_173621/1290533553.py:4: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [15]:
print(testData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 2845],
    edge_label=[2214],
    edge_label_index=[2, 2214],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 2845] }
)


In [88]:
2371 + 474

2845

In [4]:
directory_path = './NE Dataset'

In [5]:
pattern = '*.csv'

In [6]:
files = glob.glob(os.path.join(directory_path, pattern))

In [7]:
for file in files:
    print(os.path.basename(file))

val_dti.csv
drug_attr.csv
train_ddi.csv
test_dti.csv
train_ppi.csv
train_dti.csv
protein_attr.csv


In [6]:
# 1st I will load the drug attribute dataset
# then I will create a two dimensional tensor of drug features. that tensor size will be [1482, 1444]
# I will map to drugs to qnique values (0 ....... num_of_drugs - 1)

In [8]:
file_path = './NE Dataset/drug_attr.csv'

In [9]:
dfDrug = pd.read_csv(file_path)

/tmp/ipykernel_109814/782197566.py:1: DtypeWarning: Columns (663,786,787,832,835,869,905,906,951,954,988,1146,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDrug = pd.read_csv(file_path)


In [10]:
dfDrug

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445
0,0,DB01356,0,0.0000,0.000000,0.0000,24.300000,0,0,1,...,7.016005,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0,0.000,0
1,1,DB06757,0,0.0000,0.000000,0.0000,9.400000,0,0,1,...,54.938045,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0,0.000,0
2,2,DB01370,0,0.0000,0.000000,0.0000,6.800000,0,0,1,...,26.981539,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0,0.000,0
3,3,DB00435,0,-0.1269,0.016104,2.2429,2.568793,0,0,3,...,10.335271,3.000000,1.500000,4.000000,2.000000,2.0,1.0,0,0.028,2
4,4,DB03166,1,-0.2299,0.052854,12.6437,7.791172,0,0,8,...,7.502641,6.732051,1.683013,4.488034,4.488034,0.0,9.0,0,-0.080,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,1477,DB00104,0,0.0000,0.000000,0.0000,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0
1478,1478,DB00569,0,0.0000,0.000000,0.0000,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0
1479,1479,DB00093,0,0.0000,0.000000,0.0000,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0
1480,1480,DB09067,0,0.0000,0.000000,0.0000,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0


In [11]:
dfDrugFeat = dfDrug.drop(dfDrug.columns[:2], axis=1)

In [12]:
dfDrugFeat

,2,3,4,5,6,7,8,9,10,11,...,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445
0,0,0.0000,0.000000,0.0000,24.300000,0,0,1,1,0,...,7.016005,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0,0.000,0
1,0,0.0000,0.000000,0.0000,9.400000,0,0,1,1,0,...,54.938045,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0,0.000,0
2,0,0.0000,0.000000,0.0000,6.800000,0,0,1,1,0,...,26.981539,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0,0.000,0
3,0,-0.1269,0.016104,2.2429,2.568793,0,0,3,2,1,...,10.335271,3.000000,1.500000,4.000000,2.000000,2.0,1.0,0,0.028,2
4,1,-0.2299,0.052854,12.6437,7.791172,0,0,8,4,4,...,7.502641,6.732051,1.683013,4.488034,4.488034,0.0,9.0,0,-0.080,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,0,0.0000,0.000000,0.0000,0.000000,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0
1478,0,0.0000,0.000000,0.0000,0.000000,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0
1479,0,0.0000,0.000000,0.0000,0.000000,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0
1480,0,0.0000,0.000000,0.0000,0.000000,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000,0


In [13]:
dfDrugFeat['663']

0            #NAME?
1            #NAME?
2            #NAME?
3       0.069314718
4       5.965649139
           ...     
1477              0
1478              0
1479              0
1480              0
1481              0
Name: 663, Length: 1482, dtype: object

In [14]:
# replacing the missing values with 0
dfDrugFeat.replace('#NAME?', np.nan, inplace=True)
dfDrugFeat.fillna(0, inplace=True)

In [15]:
dfDrugFeat['663']

0                 0
1                 0
2                 0
3       0.069314718
4       5.965649139
           ...     
1477              0
1478              0
1479              0
1480              0
1481              0
Name: 663, Length: 1482, dtype: object

In [16]:
# dfDrugFeat dataframe has int, float and str type data in it
# replact all the values to float type
def to_float(value):
    try:
        return float(value)
    except ValueError:
        return float('nan')
   
dfDrugFeat = dfDrugFeat.applymap(to_float)

In [18]:
# dfDrugFeat

In [19]:
max_value = dfDrugFeat.max().max()
print(max_value)

inf


In [20]:
# replacing the inf value with zero
dfDrugFeat.replace([np.inf], 0, inplace=True)

In [21]:
min_value = dfDrugFeat.min().min()
max_value = dfDrugFeat.max().max()
print("Minimum: ", min_value)
print("Maximum: ", max_value)

Minimum:  -784247906.0
Maximum:  1.7976931348623157e+308


In [38]:
2048 * 2

4096

In [19]:
# NORMALIZE HERE

In [22]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [23]:
dfDrugFeat = scaler.fit_transform(dfDrugFeat)

In [24]:
dfDrugFeat.shape

(1482, 1444)

In [25]:
min_value = dfDrugFeat.min().min()
max_value = dfDrugFeat.max().max()
print("Minimum: ", min_value)
print("Maximum: ", max_value)

Minimum:  -1.0
Maximum:  1.0000000000000004


In [26]:
print(dfDrugFeat.shape)

(1482, 1444)


In [27]:
type(dfDrugFeat)

numpy.ndarray

In [29]:
# dfDrugFeat['68']

In [30]:
# the drug feature tensor of size [1482, 1444]
drug_feat = torch.from_numpy(dfDrugFeat).to(torch.float)

In [22]:
type(drug_feat)

torch.Tensor

In [23]:
max_value = torch.max(drug_feat)
print(max_value)

tensor(1.)


In [24]:
drug_feat.shape

torch.Size([1482, 1444])

In [25]:
uniqueDrug = dfDrug['1'].unique()

In [26]:
uniqueDrug = pd.DataFrame(data={
    'DrugId': uniqueDrug,
    'mappedId': pd.RangeIndex(len(uniqueDrug)),
})

In [18]:
# uniqueDrug

In [20]:
# storing this uniqueDrug dataframe in csv format in the folder 'Modified Dataset'
## RUN THIS CELL ONLY ONE TIME
# file_path_for_store = os.path.join('Modified Dataset','uniqueDrug.csv')
# uniqueDrug.to_csv(file_path_for_store, index=False)

In [30]:
# 1st I will load the target dataset
# then I will creat a 2 dimensional tensor of target features. That tensor will have size [1408, 4159]
# I will map target to unique value range [0, num_of_target - 1]

In [27]:
file_path = './NE Dataset/protein_attr.csv'

In [28]:
dfTarget = pd.read_csv(file_path)

In [31]:
# dfTarget

In [29]:
dfTargetFeat = dfTarget.drop(dfTarget.columns[:2], axis=1)

In [34]:
# dfTargetFeat

In [30]:
# target feature tensor of size [1408, 4159]
target_feat = torch.from_numpy(dfTargetFeat.values).to(torch.float)

In [31]:
target_feat.shape

torch.Size([1408, 4159])

In [32]:
uniqueTraget = dfTarget['#'].unique()

In [33]:
len(uniqueTraget)

1408

In [34]:
uniqueTraget = pd.DataFrame(data={
    'TargetId': uniqueTraget,
    'mappedId': pd.RangeIndex(len(uniqueTraget)),
})

In [27]:
# uniqueTraget

In [68]:
# storing this uniqueTraget dataframe in csv format in the folder 'Modified Dataset'
## RUN THIS CELL ONLY ONE TIME
# file_path_for_store = os.path.join('Modified Dataset','uniqueTraget.csv')
# uniqueTraget.to_csv(file_path_for_store, index=False)

In [41]:
# create the edge index
# create edge index drug to target

In [35]:
file_path = './NE Dataset/train_dti.csv'

In [36]:
dfDTI = pd.read_csv(file_path)

In [30]:
# dfDTI

In [35]:
# split the dataframe into two dataframe.
# one split will contian the postive edges
# another split will contain the negative edges

In [37]:
dfDTIpos = dfDTI[dfDTI['Label'] == 1]

In [33]:
# dfDTIpos

In [38]:
dfDTIneg = dfDTI[dfDTI['Label'] == 0]

In [36]:
# dfDTIneg

In [47]:
# create edge_index_pos_drug_to_target

In [39]:
pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
                               left_on='Drug', right_on='DrugId', how='left')
pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)

In [40]:
pos_interact_drug_id

tensor([ 787,   62,  516,  ..., 1449, 1057, 1448])

In [41]:
pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
                                  left_on='Target', right_on='TargetId', how='left')
pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)

In [58]:
# pos_interact_target_id

In [42]:
edge_index_pos_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0)

In [45]:
edge_index_pos_drug_to_target

tensor([[ 787,   62,  516,  ..., 1449, 1057, 1448],
        [ 942, 1366,  805,  ...,  913,  382,  637]])

In [35]:
# create edge_index_neg_drug_to_target

In [46]:
neg_interact_drug_id = pd.merge(dfDTIneg['Drug'], uniqueDrug,
                               left_on='Drug', right_on='DrugId', how='left')
neg_interact_drug_id = torch.from_numpy(neg_interact_drug_id['mappedId'].values)

In [47]:
neg_interact_drug_id

tensor([ 839,  490, 1126,  ...,  423, 1306,    3])

In [48]:
neg_interact_target_id = pd.merge(dfDTIneg['Target'], uniqueTraget,
                                 left_on='Target', right_on='TargetId', how='left')
neg_interact_target_id = torch.from_numpy(neg_interact_target_id['mappedId'].values)

In [49]:
neg_interact_target_id

tensor([ 667, 1054,  664,  ...,  530, 1048,  263])

In [50]:
edge_index_neg_drug_to_target = torch.stack([neg_interact_drug_id, neg_interact_target_id], dim=0)

In [52]:
edge_index_neg_drug_to_target

tensor([[ 839,  490, 1126,  ...,  423, 1306,    3],
        [ 667, 1054,  664,  ...,  530, 1048,  263]])

In [98]:
# create edge index drug to drug

In [53]:
file_path = './NE Dataset/train_ddi.csv'

In [54]:
dfDDI = pd.read_csv(file_path)

In [54]:
# dfDDI

In [55]:
source_drug_id = pd.merge(dfDDI['Drug'], uniqueDrug,
                         left_on='Drug', right_on='DrugId', how='left')
source_drug_id = torch.from_numpy(source_drug_id['mappedId'].values)

In [75]:
# source_drug_id

In [56]:
destination_drug_id = pd.merge(dfDDI['Target'], uniqueDrug,
                              left_on='Target', right_on='DrugId', how='left')
destination_drug_id = torch.from_numpy(destination_drug_id['mappedId'].values)

In [78]:
# destination_drug_id

In [117]:
edge_index_drug_to_drug = torch.stack([source_drug_id, destination_drug_id], dim=0)

In [58]:
edge_index_drug_to_drug.shape

torch.Size([2, 967801])

In [131]:
# create edge index target to target

In [59]:
file_path = './NE Dataset/train_ppi.csv'

In [60]:
dfPPI = pd.read_csv(file_path)

In [62]:
# dfPPI

In [61]:
source_target_id = pd.merge(dfPPI['Drug'], uniqueTraget,
                           left_on='Drug', right_on='TargetId', how='left')
source_target_id = torch.from_numpy(source_target_id['mappedId'].values)

In [86]:
# source_target_id

In [62]:
destination_target_id = pd.merge(dfPPI['Target'], uniqueTraget,
                                left_on='Target', right_on='TargetId', how='left')
destination_target_id = torch.from_numpy(destination_target_id['mappedId'].values)

In [63]:
destination_target_id

tensor([   1,    2,    3,  ...,  190,  854, 1243])

In [64]:
edge_index_target_to_target = torch.stack([source_target_id, destination_target_id], dim=0)

In [122]:
# edge_index_target_to_target

In [65]:
# printing the shape of all created edge index
print(edge_index_pos_drug_to_target.shape)
print(edge_index_neg_drug_to_target.shape)
print(edge_index_drug_to_drug.shape)
print(edge_index_target_to_target.shape)

torch.Size([2, 2371])
torch.Size([2, 2371])
torch.Size([2, 967801])
torch.Size([2, 1276992])


In [55]:
# we have to create heterogeneous graph
# so far we have some of the things ready
# we have drug features
# we have target features
# we have three kinds of edges: ('drug', 'interact', 'target'), ('drug', 'similar', 'drug'), ('target', 'similar', 'target')
# TO DO now
# dfDTI er pos label gulu 1D tensor a convert
# dfDTI er neg label gulu 1D tensor a convert
# pos edge_index_pos_drug_to_target split -> 70% and 30%. pos label split -> 70% and 30%
# pos edge_index_neg_drug_to_target split -> 70% and 30%. neg label split -> 70% and 30%
# 30% edge_index_pos_drug_to_target and 30% edge_index_neg_drug_to_target concatenate (this is supervision edge index)
# 30% pos label + 30% neg label (this is supervision label)

In [66]:
# dfDTI er pos label gulu 1D tensor a convert
pos_edge_label = dfDTIpos['Label']
pos_edge_label = torch.tensor(pos_edge_label.values)

In [67]:
pos_edge_label

tensor([1, 1, 1,  ..., 1, 1, 1])

In [68]:
# dfDTI er neg label gulu 1D tensor a convert
neg_edge_label = dfDTIneg['Label']
neg_edge_label = torch.tensor(neg_edge_label.values)

In [69]:
neg_edge_label

tensor([0, 0, 0,  ..., 0, 0, 0])

In [70]:
def split_edges(edge_index, edge_label, split_ratio=0.7):
    num_edges = edge_index.size(1)
    random_permutation = torch.randperm(num_edges)
    split_edge = int(split_ratio * num_edges)
    edge_index_meg_pass = edge_index[:, random_permutation[:split_edge]]
    edge_index_supervision = edge_index[:, random_permutation[split_edge: ]]
    edge_label_supervision = edge_label[random_permutation[split_edge: ]]
    return edge_index_meg_pass, edge_index_supervision, edge_label_supervision

In [71]:
edge_index_meg_pass, pos_edge_index_supervision, pos_edge_label_supervision = split_edges(edge_index_pos_drug_to_target, pos_edge_label)

In [72]:
print(edge_index_meg_pass.shape)
print(pos_edge_index_supervision.shape)
print(pos_edge_label_supervision.shape)

torch.Size([2, 1659])
torch.Size([2, 712])
torch.Size([712])


In [73]:
dropped_edge_index, neg_edge_index_supervision, neg_edge_label_supervision = split_edges(edge_index_neg_drug_to_target, neg_edge_label)

In [74]:
# print(dropped_edge_index.shape)
print(neg_edge_index_supervision.shape)
print(neg_edge_label_supervision.shape)

torch.Size([2, 712])
torch.Size([712])


In [75]:
edge_index_supervision = torch.cat((pos_edge_index_supervision, neg_edge_index_supervision), dim=1)

In [76]:
edge_label_supervision = torch.cat((pos_edge_label_supervision, neg_edge_label_supervision), dim=0)

In [77]:
edge_index_supervision

tensor([[1016,  278,  772,  ...,  313,  870,  678],
        [1315, 1003,  910,  ...,  791,  958,  762]])

In [78]:
edge_label_supervision

tensor([1, 1, 1,  ..., 0, 0, 0])

In [79]:
print(edge_index_supervision.shape)
print(edge_label_supervision.shape)

torch.Size([2, 1424])
torch.Size([1424])


In [80]:
# shuffle this supervision edge index and label
random_permutation = torch.randperm(1424)
edge_index_supervision = edge_index_supervision[:, random_permutation]
edge_label_supervision = edge_label_supervision[random_permutation]

In [81]:
edge_index_supervision

tensor([[ 886, 1135, 1362,  ...,  334,  601,  110],
        [1081,  632,  962,  ..., 1013,  719,  909]])

In [82]:
edge_label_supervision

tensor([0, 0, 1,  ..., 0, 0, 0])

In [83]:
print(edge_index_supervision.shape)
print(edge_label_supervision.shape)

torch.Size([2, 1424])
torch.Size([1424])


In [84]:
# things created so far
print(drug_feat.shape)
print(target_feat.shape)
print(edge_index_drug_to_drug.shape) # this will be used for message passing
print(edge_index_target_to_target.shape) # this will be used for message passing
print(edge_index_meg_pass.shape) # this will be used for message passing
# above three edges will be added to the heterodata and then T.ToUndirected() will be applied
# then concatenated supervision edge index and labels will be added to the heterodata
print(edge_index_supervision.shape) # this is the concatenated edge_index
print(edge_label_supervision.shape) # this is the concatenated edge_label

torch.Size([1482, 1444])
torch.Size([1408, 4159])
torch.Size([2, 967801])
torch.Size([2, 1276992])
torch.Size([2, 1659])
torch.Size([2, 1424])
torch.Size([1424])


In [88]:
# creating trainData

In [85]:
trainData = HeteroData()

In [86]:
# adding node and features of drug
trainData['drug'].node_id = torch.arange(drug_feat.size(0))
trainData['drug'].x = drug_feat

In [87]:
# adding node and features of target
trainData['target'].node_id = torch.arange(target_feat.size(0))
trainData['target'].x = target_feat

In [89]:
# trainData['target']

In [88]:
# adding all the edges which will be used for message passing to the heterogeneous data
trainData["drug", "interact", "target"].edge_index = edge_index_meg_pass
trainData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
trainData["target", "similar", "target"].edge_index = edge_index_target_to_target

In [89]:
trainData.metadata()

(['drug', 'target'],
 [('drug', 'interact', 'target'),
  ('drug', 'similar', 'drug'),
  ('target', 'similar', 'target')])

In [90]:
print(trainData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={ edge_index=[2, 1659] },
  (drug, similar, drug)={ edge_index=[2, 967801] },
  (target, similar, target)={ edge_index=[2, 1276992] }
)


In [91]:
trainData = T.ToUndirected()(trainData)

In [92]:
print(trainData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={ edge_index=[2, 1659] },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 1659] }
)


In [93]:
trainData.metadata()

(['drug', 'target'],
 [('drug', 'interact', 'target'),
  ('drug', 'similar', 'drug'),
  ('target', 'similar', 'target'),
  ('target', 'rev_interact', 'drug')])

In [94]:
# adding edge index and edge label for supervision to the data
trainData["drug", "interact", "target"].edge_label = edge_label_supervision
trainData["drug", "interact", "target"].edge_label_index = edge_index_supervision

In [95]:
print(trainData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 1659],
    edge_label=[1424],
    edge_label_index=[2, 1424],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 1659] }
)


In [96]:
trainData.metadata()

(['drug', 'target'],
 [('drug', 'interact', 'target'),
  ('drug', 'similar', 'drug'),
  ('target', 'similar', 'target'),
  ('target', 'rev_interact', 'drug')])

In [97]:
trainData["drug", "interact", "target"]

{'edge_index': tensor([[ 340,  271,  699,  ...,  134,  361, 1328],
        [ 507,    9,  507,  ...,  494, 1011,  664]]), 'edge_label': tensor([0, 0, 1,  ..., 0, 0, 0]), 'edge_label_index': tensor([[ 886, 1135, 1362,  ...,  334,  601,  110],
        [1081,  632,  962,  ..., 1013,  719,  909]])}

In [98]:
trainData.edge_index_dict

{('drug',
  'interact',
  'target'): tensor([[ 340,  271,  699,  ...,  134,  361, 1328],
         [ 507,    9,  507,  ...,  494, 1011,  664]]),
 ('drug',
  'similar',
  'drug'): tensor([[   2,    3,    3,  ..., 1481, 1481, 1481],
         [1435,    4,    5,  ..., 1477, 1478, 1480]]),
 ('target',
  'similar',
  'target'): tensor([[   0,    0,    0,  ..., 1398, 1398, 1398],
         [   1,    2,    3,  ..., 1241, 1242, 1340]]),
 ('target',
  'rev_interact',
  'drug'): tensor([[ 507,    9,  507,  ...,  494, 1011,  664],
         [ 340,  271,  699,  ...,  134,  361, 1328]])}

In [99]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # self.conv1 = GCNConv(hidden_channels, hidden_channels)
        # self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
#         self.conv1 = GATConv(hidden_channels, hidden_channels, add_self_loops=True)
#         self.conv2 = GATConv(hidden_channels, hidden_channels, add_self_loops=True)
    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [100]:
class Classifier(torch.nn.Module):
    def forward(self, x_drug: Tensor, x_target: Tensor, edge_label_index: Tensor):
        edge_feat_drug = x_drug[edge_label_index[0]]
        edge_feat_target = x_target[edge_label_index[1]]
#         dotProduct = (edge_feat_drug * edge_feat_target).sum(dim=1)
#         print("Dot Product: ", dotProduct)
#         print(dotProduct.size())
        return (edge_feat_drug * edge_feat_target).sum(dim=1)

In [101]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.drug_lin = torch.nn.Linear(1444, hidden_channels)
        self.target_lin = torch.nn.Linear(4159, hidden_channels)
        # Instantiate homogeneous GNN
        self.gnn = GNN(hidden_channels)
        # Convert GNN model into a heterogeneous variant
        self.gnn = to_hetero(self.gnn, metadata=trainData.metadata())
        self.classifier = Classifier()
    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
            "drug": self.drug_lin(data["drug"].x),
            "target": self.target_lin(data["target"].x),
        
        }
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["drug"],
            x_dict["target"],
            data["drug", "interact", "target"].edge_label_index,
        )
        return pred

In [103]:
model = Model(hidden_channels=64)

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

Device: 'cpu'


In [105]:
model = model.to(device)

In [106]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [107]:
ground_truth = trainData["drug", "interact", "target"].edge_label
ground_truth = ground_truth.float()
ground_truth.to(device)

tensor([0., 0., 1.,  ..., 0., 0., 0.])

In [108]:
print(ground_truth.device)

cpu


In [109]:
for epoch in range(1, 101):
    optimizer.zero_grad()
    pred = model(trainData)
    loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")
    loss.backward()
    optimizer.step()

Epoch: 10, Loss: 225.79443359375
Epoch: 20, Loss: 11.151211738586426
Epoch: 30, Loss: 13.802911758422852
Epoch: 40, Loss: 4.4989213943481445
Epoch: 50, Loss: 2.128450632095337
Epoch: 60, Loss: 1.1115609407424927
Epoch: 70, Loss: 0.795632004737854
Epoch: 80, Loss: 0.6876006126403809
Epoch: 90, Loss: 0.615020215511322
Epoch: 100, Loss: 0.5835761427879333


In [113]:
# creating the validation dataset

In [120]:
valData = HeteroData()

In [123]:
# adding node and features of drug
valData['drug'].node_id = torch.arange(drug_feat.size(0))
valData['drug'].x = drug_feat

In [124]:
# adding node and features of target
valData['target'].node_id = torch.arange(target_feat.size(0))
valData['target'].x = target_feat

In [125]:
print(valData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  }
)


In [204]:
# dfDTI

In [114]:
# pos_interact_drug_id = pd.merge(dfDTIpos['Drug'], uniqueDrug,
#                                left_on='Drug', right_on='DrugId', how='left')
# pos_interact_drug_id = torch.from_numpy(pos_interact_drug_id['mappedId'].values)

In [115]:
# pos_interact_target_id = pd.merge(dfDTIpos['Target'], uniqueTraget,
#                                   left_on='Target', right_on='TargetId', how='left')
# pos_interact_target_id = torch.from_numpy(pos_interact_target_id['mappedId'].values)

In [116]:
# edge_index_drug_to_target = torch.stack([pos_interact_drug_id, pos_interact_target_id], dim=0)

In [126]:
edge_index_pos_drug_to_target

tensor([[ 787,   62,  516,  ..., 1449, 1057, 1448],
        [ 942, 1366,  805,  ...,  913,  382,  637]])

In [119]:
# edge_index_drug_to_target

tensor([[ 787,   62,  516,  ..., 1449, 1057, 1448],
        [ 942, 1366,  805,  ...,  913,  382,  637]])

In [127]:
# adding all the edges which will be used for message passing to the heterogeneous data
valData["drug", "interact", "target"].edge_index = edge_index_pos_drug_to_target
valData["drug", "similar", "drug"].edge_index = edge_index_drug_to_drug
valData["target", "similar", "target"].edge_index = edge_index_target_to_target

In [128]:
file_path = './NE Dataset/val_dti.csv'

In [129]:
dfvalDTI = pd.read_csv(file_path)

In [130]:
dfvalDTI

,Unnamed: 0,Drug,Target,Label
0,0,DB01567,A8MPY1,1
1,1,DB01092,P46721,1
2,2,DB00281,P10632,1
3,3,DB01120,P15692,1
4,4,DB08896,Q9UNQ0,1
...,...,...,...,...
943,943,DB01559,P30542,0
944,944,DB01336,P08183,0
945,945,DB00451,P21397,0
946,946,DB01410,P08684,0


In [131]:
edge_index_supervision_drug = pd.merge(dfvalDTI['Drug'], uniqueDrug,
                                      left_on='Drug', right_on='DrugId', how='left')
edge_index_supervision_drug = torch.from_numpy(edge_index_supervision_drug['mappedId'].values)

In [132]:
edge_index_supervision_target = pd.merge(dfvalDTI['Target'], uniqueTraget,
                                        left_on='Target', right_on='TargetId', how='left')
edge_index_supervision_target = torch.from_numpy(edge_index_supervision_target['mappedId'].values)

In [133]:
edge_index_supervision = torch.stack([edge_index_supervision_drug, edge_index_supervision_target], dim=0)

In [134]:
edge_index_supervision.size()

torch.Size([2, 948])

In [135]:
edge_label_supervision = dfvalDTI['Label']
edge_label_supervision = torch.tensor(edge_label_supervision.values)

In [196]:
# print(edge_index_supervision)
# print(edge_label_supervision)

In [136]:
random_permutationVal = torch.randperm(948)

In [137]:
edge_index_supervision = edge_index_supervision[:, random_permutationVal]
edge_label_supervision = edge_label_supervision[random_permutationVal]

In [197]:
# print(edge_index_supervision)
# print(edge_label_supervision)

In [180]:
# valData.metadata()
# print(valData)

In [220]:
valData = T.ToUndirected()(valData)

In [184]:
# valData.metadata()
# print(valData)

In [221]:
# adding edge index and edge label for supervision to the data
valData["drug", "interact", "target"].edge_label = edge_label_supervision
valData["drug", "interact", "target"].edge_label_index = edge_index_supervision

In [222]:
print(valData)

HeteroData(
  drug={
    node_id=[1482],
    x=[1482, 1444],
  },
  target={
    node_id=[1408],
    x=[1408, 4159],
  },
  (drug, interact, target)={
    edge_index=[2, 2371],
    edge_label=[948],
    edge_label_index=[2, 948],
  },
  (drug, similar, drug)={ edge_index=[2, 1246640] },
  (target, similar, target)={ edge_index=[2, 1043321] },
  (target, rev_interact, drug)={ edge_index=[2, 2371] }
)
